## Integrating Datasets and Engineering Features

This notebook contains examples and code from various sources:

<a href="github.com/alicezheng/feature-engineering-book">Feature Engineering for Machine Learning Book Code Repository </a>
<br><a href="github.com/jakevdp/PythonDataScienceHandbook"> Python Data Science Handbook Code Repository </a>

The goal of this notebook is to illustrate how integration of multiple datasets can be done and how features can be engineered.

Detailed explanations for important code snippets are provided by Mervat Abuelkheir as part of the CSEN1095 Data Engineering Course.

Pay attention to the <span style="color:red"> <b> paragraphs in bold red</b></span>; they ask you to do something and provide input!


For convenience, we will start by redefining the `display()` functionality from the previous section:


In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
%matplotlib inline

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Part 1: Merging and Joining Datasets

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.
The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.

### Relational Algebra

The behavior implemented in ``pd.merge()`` is a subset of what is known as *relational algebra*, which is a formal set of rules for manipulating relational data, and forms the conceptual foundation of operations available in most databases.
The strength of the relational algebra approach is that it proposes several primitive operations, which become the building blocks of more complicated operations on any dataset.
With this lexicon of fundamental operations implemented efficiently in a database or other program, a wide range of fairly complicated composite operations can be performed.

Pandas implements several of these fundamental building-blocks in the ``pd.merge()`` function and the related ``join()`` method of ``Series`` and ``Dataframe``s.
As we will see, these let you efficiently link data from different sources.

### Categories of Joins

The ``pd.merge()`` function implements a number of types of joins: the *one-to-one*, *many-to-one*, and *many-to-many* joins.
All three types of joins are accessed via an identical call to the ``pd.merge()`` interface; the type of join performed depends on the form of the input data.
Here we will show simple examples of the three types of merges, and discuss detailed options further below.

#### One-to-one joins

Perhaps the simplest type of merge expresion is the one-to-one join, which is in many ways very similar to the column-wise concatenation.

As a concrete example, consider the following two ``DataFrames`` which contain information on several employees in a company:

In [2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


To combine this information into a single ``DataFrame``, we can use the ``pd.merge()`` function. 

The ``pd.merge()`` function recognizes that each ``DataFrame`` has an "employee" column, and automatically joins using this column as a key.
The result of the merge is a new ``DataFrame`` that combines the information from the two inputs.
Notice that the order of entries in each column is not necessarily maintained: in this case, the order of the "employee" column differs between ``df1`` and ``df2``, and the ``pd.merge()`` function correctly accounts for this.

Additionally, keep in mind that the merge in general discards the index, except in the special case of merges by index (see the ``left_index`` and ``right_index`` keywords, discussed momentarily).

In [3]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### Many-to-one joins

Many-to-one joins are joins in which one of the two key columns contains duplicate entries.
For the many-to-one case, the resulting ``DataFrame`` will preserve those duplicate entries as appropriate. 
Consider the following example of a many-to-one join:

In [4]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

The resulting ``DataFrame`` will have an aditional column with the "supervisor" information, where the information is repeated in one or more locations as required by the inputs.

#### Many-to-many joins

Many-to-many joins are a bit confusing conceptually, but are nevertheless well defined.
If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge.
This will be perhaps most clear with a concrete example.
Consider the following, where we have a ``DataFrame`` showing one or more skills associated with a particular group.
By performing a many-to-many join, we can recover the skills associated with any individual person:

In [5]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


 In practice, datasets are rarely as clean as the one we worked with above. In the following section we'll consider some of the options provided by `pd.merge()` that enable you to tune how the join operations work.

### Specification of the Merge Key

We've already seen the default behavior of ``pd.merge()``: it looks for one or more matching column names between the two inputs, and uses this as the key.
However, often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

#### The ``on`` keyword

Most simply, you can explicitly specify the name of the key column using the ``on`` keyword, which takes a column name or a list of column names. This option works only if both the left and right ``DataFrame``s have the specified column name.

In [6]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

#### The ``left_on`` and ``right_on`` keywords

At times you may wish to merge two datasets with different column names; for example, we may have a dataset in which the employee name is labeled as "name" rather than "employee".
In this case, we can use the ``left_on`` and ``right_on`` keywords to specify the two column names:

In [7]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

The result has a redundant column that we can drop if desired–for example, by using the ``drop()`` method of ``DataFrame``s:

In [8]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


#### The ``left_index`` and ``right_index`` keywords

Sometimes, rather than merging on a column, you would instead like to merge on an index.
For example, your data might look like this:

In [9]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


You can use the index as the key for merging by specifying the ``left_index`` and/or ``right_index`` flags in ``pd.merge()``:

In [10]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


For convenience, ``DataFrame``s implement the ``join()`` method, which performs a merge that defaults to joining on indices:

In [11]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


### Specifying Set Arithmetic for Joins

In all the preceding examples we have glossed over one important consideration in performing a join: the type of set arithmetic used in the join.
This comes up when a value appears in one key column but not the other. Consider this example:

In [12]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Here we have merged two datasets that have only a single "name" entry in common: Mary.
By default, the result contains the *intersection* of the two sets of inputs; this is what is known as an *inner join*.
We can specify this explicitly using the ``how`` keyword, which defaults to ``"inner"``:

In [13]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Other options for the ``how`` keyword are ``'outer'``, ``'left'``, and ``'right'``.
An *outer join* returns a join over the union of the input columns, and fills in all missing values with NAs:

In [14]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

The *left join* and *right join* return joins over the left entries and right entries, respectively.
For example:

In [15]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

The output rows now correspond to the entries in the left input. Using
``how='right'`` works in a similar manner.

### Overlapping Column Names: The ``suffixes`` Keyword

Finally, you may end up in a case where your two input ``DataFrame``s have conflicting column names.
Consider this example:

In [16]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

## Example: The US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found in the data folder.

Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [17]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

In [18]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # drop duplicate info
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [19]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Some of the ``population`` info is null; let's figure out which these are!

In [20]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [21]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [22]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No more nulls in the ``state`` column: we're all set!

Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the ``state`` column in both:

In [23]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Again, let's check for nulls to see if there were any mismatches:

In [24]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

There are nulls in the ``area`` column; we can take a look to see which regions were ignored here:

In [25]:
#final['state'][final['area (sq. mi)'].isnull()].unique()
final.loc[final['area (sq. mi)'].isnull(), 'state'].unique()

array(['United States'], dtype=object)

We see that our ``areas`` ``DataFrame`` does not contain the area of the United States as a whole.
We could insert the appropriate value (using the sum of all state areas, for instance), but in this case we'll just drop the null values because the population density of the entire United States is not relevant to our current discussion:

In [26]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Let's check if there are any duplicate records in the dataset before we proceed to query it.

In [27]:
final.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2491    False
2492    False
2493    False
2494    False
2495    False
Length: 2476, dtype: bool

Thankfully there are no duplicates. Now we have all the data we need. To answer the question of interest, let's first select the portion of the data corresponding with the year 2000, and the total population.
We'll use the ``query()`` function to do this quickly (this requires the ``numexpr`` package to be installed.

In [28]:
!pip install numexpr

In [29]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Now let's compute the population density and display it in order.
We'll start by re-indexing our data on the state, and then compute the result:

In [30]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [31]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [32]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

## <span style="color:red">Assignment Mini Challenge 1</span>

In the data/microbiome subdirectory, there are 9 spreadsheets of microbiome data that was acquired from RNA sequencing procedures, with a 10th file describing the content of each spreadsheet. 

<span style="color:red"><b>Write code that imports each of the data spreadsheets and combines them into a single `DataFrame`, adding identifying information from the metadata spreadsheet as columns in the combined `DataFrame`.</b></span>

In [42]:
MID1 = pd.read_excel('data\microbiome\MID1.xls')
MID2 = pd.read_excel('data\microbiome\MID2.xls')
MID3 = pd.read_excel('data\microbiome\MID3.xls')
MID4 = pd.read_excel('data\microbiome\MID4.xls')
MID5 = pd.read_excel('data\microbiome\MID5.xls')
MID6 = pd.read_excel('data\microbiome\MID6.xls')
MID7 = pd.read_excel('data\microbiome\MID7.xls')
MID8 = pd.read_excel('data\microbiome\MID8.xls')
MID9 = pd.read_excel('data\microbiome\MID9.xls')

MID1.columns=["name","EXTRACTION CONTROL_Na"]
MID2.columns=["name","NEC 1_tissue"]
MID3.columns=["name","Control 1_tissue"]
MID4.columns=["name","NEC 2_tissue"]
MID5.columns=["name","Control 2_tissue"]
MID6.columns=["name","NEC 1_stool"]
MID7.columns=["name","Control 1_stool"]
MID8.columns=["name","NEC 2_stool"]
MID9.columns=["name","Control 2_stool"]

merged = pd.merge(MID1, MID2, how='outer')
merged = pd.merge(merged, MID3, how='outer')
merged = pd.merge(merged, MID4, how='outer')
merged = pd.merge(merged, MID5, how='outer')
merged = pd.merge(merged, MID6, how='outer')
merged = pd.merge(merged, MID7, how='outer')
merged = pd.merge(merged, MID8, how='outer')
merged = pd.merge(merged, MID9, how='outer')

merged.head()


,name,EXTRACTION CONTROL_Na,NEC 1_tissue,Control 1_tissue,NEC 2_tissue,Control 2_tissue,NEC 1_stool,Control 1_stool,NEC 2_stool,Control 2_stool
0,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",2.0,2.0,NaN,NaN,3.0,2.0,1.0,NaN,NaN
1,"Archaea ""Crenarchaeota"" Thermoprotei Sulfoloba...",3.0,10.0,4.0,NaN,14.0,5.0,5.0,NaN,6.0
2,"Archaea ""Crenarchaeota"" Thermoprotei Thermopro...",3.0,9.0,5.0,NaN,10.0,4.0,5.0,NaN,5.0
3,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",7.0,9.0,7.0,1.0,17.0,12.0,18.0,NaN,14.0
4,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",1.0,12.0,2.0,NaN,11.0,1.0,2.0,NaN,6.0


### Checking for categorical variables independence using the Chi-Square test

To illustrate how to use the chi-square test to determine correlation between categorical attributes, let's use another dataset that has more potential for correlated attributes: the Census (Adult Income) Dataset. 

### The Adult Income Dataset

The <a href="https://www.kaggle.com/wenruliu/adult-income-dataset">Adult Income Dataset</a> includes data about an individual’s annual income. Intuitively, income is influenced by the individual’s education level, age, gender, occupation, and etc. The dataset contains 14 columns detailing attributes related to the demographics and other features that describe a person. The target attribute, Income, is divide into two classes: <=50K and >50K. A description of the attributes follows:

<b>age</b>: continuous.
<br><b>workclass</b>: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
<br><b>fnlwgt</b>: continuous.
<br><b>education</b>: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
<br><b>education-num</b>: continuous.
<br><b>marital-status</b>: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
<br><b>occupation</b>: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
<br><b>relationship</b>: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
<br><b>race</b>: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
<br><b>gender</b>: Female, Male.
<br><b>capital-gain</b>: continuous.
<br><b>capital-loss</b>: continuous.
<br><b>hours-per-week</b>: continuous.
<br><b>native-country</b>: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
<br><b>income</b>: >50K, <=50K

Let's import some important modules and then import the data.

In [46]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

income_df = pd.read_csv("data/income_data.csv")

# List attribute names
print (list(income_df[1]))

# Display the shape of the dataset (#rows, #columns)
print (income_df.shape)

['age', 'workclass', 'fnlwgt', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
(48842, 15)


You may want to inspect some records from the dataset to get a feel of the attributes and their values.

In [47]:
#income_df.head() # first 5 rows
income_df.sample(10) # random 10 rows

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
12121,50,Private,151159,Assoc-voc,11,Married-civ-spouse,Other-service,Husband,White,Male,0,0,55,United-States,<=50K
9979,32,Private,266345,Some-college,10,Never-married,Tech-support,Not-in-family,White,Male,0,0,40,United-States,<=50K
12740,22,Private,193190,HS-grad,9,Never-married,Other-service,Own-child,Black,Female,0,0,40,United-States,<=50K
4180,53,Private,76128,HS-grad,9,Divorced,Craft-repair,Not-in-family,Other,Male,0,0,60,Ecuador,<=50K
749,49,Federal-gov,252616,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,Asian-Pac-Islander,Male,0,0,40,Philippines,<=50K
31194,61,Private,339358,5th-6th,3,Married-civ-spouse,Farming-fishing,Other-relative,White,Female,0,0,45,Mexico,<=50K
42400,18,?,344742,10th,6,Never-married,?,Own-child,White,Male,0,0,40,United-States,<=50K
14300,42,Private,234508,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K
29511,20,Private,691830,HS-grad,9,Never-married,Sales,Own-child,Black,Female,0,0,35,United-States,<=50K
42069,18,Private,170627,11th,7,Never-married,Other-service,Own-child,White,Female,0,0,15,United-States,<=50K


Let's explore if there is a correlation with gender and workclass. First, we need to build the contingency matrix for the two attributes:

In [36]:
contengency_table = pd.crosstab(income_df["workclass"],income_df["gender"], margins= True)
contengency_table

gender,Female,Male,All
workclass,,,
?,1270,1529,2799
Federal-gov,452,980,1432
Local-gov,1258,1878,3136
Never-worked,3,7,10
Private,11599,22307,33906
Self-emp-inc,211,1484,1695
Self-emp-not-inc,629,3233,3862
State-gov,763,1218,1981
Without-pay,7,14,21


Let's state the hypotheses:

<b>Null hypothesis:</b> There is no statistically significant relationship between gender and workclass.

<b>Alternative hypothesis:</b> There is a statistically significant relationship between the gender and workclass.

Each cell in the table represents the frequency count for the intersection of both values. The intersection of "male" and "federal-gov" represents the number of men who work in jobs related to the federal government.

Now let's calculate the chi-square test using SciPy. The `chi2_contengency()` method is applied to a two dimensional array representing the actual attribute values, and it automatically computes the contingency matrix and outputs three numbers: the chi2 value (difference between observed and expected counts), the p-value, and the degrees of freedom. What you want to check is the second value outputted, which is the p-value. You want it to be smaller than 0.05 so that the null hypothesis can be rejected.   

In [48]:
# Take the previously produced contingency matrix and apply the chi2 test method to it
#st.chi2_contingency(_)
st.chi2_contingency(contengency_table)

(1125.2572181055543,
 1.137527721203814e-227,
 18,
 array([[9.27918758e+02, 1.87108124e+03, 2.79900000e+03],
        [4.74733713e+02, 9.57266287e+02, 1.43200000e+03],
        [1.03964031e+03, 2.09635969e+03, 3.13600000e+03],
        [3.31517956e+00, 6.68482044e+00, 1.00000000e+01],
        [1.12404478e+04, 2.26655522e+04, 3.39060000e+04],
        [5.61922935e+02, 1.13307706e+03, 1.69500000e+03],
        [1.28032235e+03, 2.58167765e+03, 3.86200000e+03],
        [6.56737071e+02, 1.32426293e+03, 1.98100000e+03],
        [6.96187707e+00, 1.40381229e+01, 2.10000000e+01],
        [1.61920000e+04, 3.26500000e+04, 4.88420000e+04]]))

<span style="color:red"><b>Do we reject the null hypothesis based on the previous results?</b></span>

<span style="color:red"><b>In the previous output, and based on your understanding of the chi2 test from the lecture, are the degrees of freedom produced correct? If no, explain why and fix the problem and recompute the chi2 test.</b></span>


## <span style="color:red">Assignment Mini Challenge 2</span>

<span style="color:red"><b>Write code that will compute the correlation between the `race` and `education` attributes. Is there a correlation or are the attributes independent?</b></span>

In [51]:
contengency_table = pd.crosstab(income_df["race"],income_df["education"], margins= True)
# Take the previously produced contingency matrix and apply the chi2 test method to it
#st.chi2_contingency(_)
st.chi2_contingency(contengency_table)


(1040.6622432490265,
 4.8101145033105166e-167,
 80,
 array([[1.33661603e+01, 1.74366324e+01, 6.32222268e+00, 2.37684779e+00,
         4.89803857e+00, 9.18983662e+00, 7.27488637e+00, 1.54062078e+01,
         1.98327259e+01, 7.72234962e+01, 5.71598215e+00, 1.51887310e+02,
         2.55679538e+01, 7.98697842e-01, 8.02546988e+00, 1.04677532e+02,
         4.70000000e+02],
        [4.31982925e+01, 5.63537120e+01, 2.04328856e+01, 7.68176979e+00,
         1.58300438e+01, 2.97007698e+01, 2.35118136e+01, 4.97915524e+01,
         6.40976823e+01, 2.49579767e+02, 1.84735678e+01, 4.90886860e+02,
         8.26334507e+01, 2.58132345e+00, 2.59376356e+01, 3.38308874e+02,
         1.51900000e+03],
        [1.33235023e+02, 1.73809836e+02, 6.30204537e+01, 2.36926211e+01,
         4.88240654e+01, 9.16050735e+01, 7.25166865e+01, 1.53570390e+02,
         1.97694300e+02, 7.69770382e+02, 5.69773965e+01, 1.51402563e+03,
         2.54863540e+02, 7.96148806e+00, 7.99985668e+01, 1.04343454e+03,
         4.68500000e

## Part 2 - Engineering Features






Feature engineering is not a deterministic task, but some prelimenary things can still be done in a straigtforward way. In the following cells we will perform encoding, discretization, and aggregation tasks as part of feature engineering.

We are still using the Adult Income Dataset. Let's investigate descriptions of individual attributes.

In [52]:
def summerize_data(df):
    for column in df.columns:
        print (column)
        if df.dtypes[column] == np.object: # Categorical data
            print (df[column].value_counts())
        else:
            print (df[column].describe()) 
            
        print ('\n')
    
summerize_data(income_df)

age
count    48842.000000
mean        38.643585
std         13.710510
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64


workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
?                    2799
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: workclass, dtype: int64


fnlwgt
count    4.884200e+04
mean     1.896641e+05
std      1.056040e+05
min      1.228500e+04
25%      1.175505e+05
50%      1.781445e+05
75%      2.376420e+05
max      1.490400e+06
Name: fnlwgt, dtype: float64


education
HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6t

You can use the `OneHotCategoricalEncoder()` method to perform one-hot encoding, but you need to install the feature-engine package using:

    pip install feature-engine

at the command prompt. Let's try this method.

In [55]:
!pip install feature-engine

In [56]:
# Try one hot encoding

# Copy the original data
encoded_income_df = income_df.copy()

# Select the numeric columns
numeric_subset = income_df.select_dtypes('number')
# Select the categorical columns
categorical_subset = income_df.select_dtypes('object')
# Import feature-engine
from feature_engine.categorical_encoders import OneHotCategoricalEncoder

# Define encoder method
encoder = OneHotCategoricalEncoder()

# Apply encoder to categorical subset, except income
onehot_categorical_subset = encoder.fit_transform(categorical_subset[categorical_subset.columns.drop("income")])

# Beauty of feature-engine encoder method is it already returns a dataframe, so we need not worry about conversions
# Concatenate and reconstruct new dataset
onehot_encoded_data = pd.concat([numeric_subset, onehot_categorical_subset, income_df["income"]], axis = 1)

# display sample 10 records
onehot_encoded_data.sample(10)

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Local-gov,workclass_?,workclass_Self-emp-not-inc,...,native-country_Ecuador,native-country_Yugoslavia,native-country_Hungary,native-country_Hong,native-country_Greece,native-country_Trinadad&Tobago,native-country_Outlying-US(Guam-USVI-etc),native-country_France,native-country_Holand-Netherlands,income
27749,56,238582,10,0,0,41,0,1,0,0,...,0,0,0,0,0,0,0,0,0,>50K
38814,48,125892,5,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
31098,22,105043,10,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
11138,50,107581,9,0,0,50,0,0,0,1,...,0,0,0,0,0,0,0,0,0,<=50K
12840,42,179533,9,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,0,0,>50K
30550,34,28568,9,0,0,60,0,0,0,1,...,0,0,0,0,0,0,0,0,0,>50K
26855,37,59313,10,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
8545,50,151143,9,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
12723,32,117444,13,0,0,50,1,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K
3468,40,117523,13,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,0,0,<=50K


#### Discretization 

Now let's discretize some of the numerical attributes. We will work with the <b>age</b> and the <b>hours-per-week</b> as examples.

In [57]:
# Group the "age" column
age_group = [] # define array structure
for age in encoded_income_df["age"]:
    if age < 25:
        age_group.append("<25")
    elif 25 <= age <= 34:
        age_group.append("25-34")
    elif 34 < age <= 44:
        age_group.append("35-44")
    elif 44 < age <= 54:
        age_group.append("45-54")
    elif 54 < age <= 65:
        age_group.append("55-64")
    else:
        age_group.append("65 and over")
        
# Copy dataframe to keep original 
new_income_df = encoded_income_df.copy()
new_income_df["age_group"] = age_group
del new_income_df["age"]

# Same thing for "hours-per-week"
work_hours_per_week = []
for hours in encoded_income_df["hours-per-week"]:
    if hours < 16:
        work_hours_per_week.append("<16")
    elif 16 <= hours <= 32:
        work_hours_per_week.append("16-32")
    elif 32 < hours <= 48:
        work_hours_per_week.append("32-48")
    elif 48 < hours <= 60:
        work_hours_per_week.append("48-60")
    else:
        work_hours_per_week.append("60 and over")
        
new_income_df["work_hours_per_week"] = work_hours_per_week
del new_income_df["hours-per-week"]

new_income_df.head(10)

,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,native-country,income,age_group,work_hours_per_week
0,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,United-States,<=50K,25-34,32-48
1,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,United-States,<=50K,35-44,48-60
2,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,United-States,>50K,25-34,32-48
3,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,United-States,>50K,35-44,32-48
4,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,United-States,<=50K,<25,16-32
5,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,United-States,<=50K,25-34,16-32
6,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,United-States,<=50K,25-34,32-48
7,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,United-States,>50K,55-64,16-32
8,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,United-States,<=50K,<25,32-48
9,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,United-States,<=50K,55-64,<16


There is an easier way in python than using if else statements. Pandas `cut` function can be used to group continuous or countable data in to bins.

In [58]:
# Take another copy of the original dataset
new_income_df2 = encoded_income_df.copy()

# Cut the age attribute into intervals
age_group2 = pd.cut(new_income_df2.age, [20,40,60,80])

# You can label the intervals for more meaningful representation
#age_group2 = pd.cut(new_income_df2.age, [20,40,60,80],labels=['young','middle-aged','old','senior'])

new_income_df2["age_group"] = age_group2
del new_income_df2["age"]

new_income_df2.head(10)

,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,age_group
0,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,"(20.0, 40.0]"
1,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,"(20.0, 40.0]"
2,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,"(20.0, 40.0]"
3,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,"(40.0, 60.0]"
4,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K,NaN
5,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,"(20.0, 40.0]"
6,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K,"(20.0, 40.0]"
7,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,"(60.0, 80.0]"
8,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K,"(20.0, 40.0]"
9,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K,"(40.0, 60.0]"


## <span style="color:red">Assignment Mini Challenge 3</span>

It is possible to use `cut` with quantiles instead of intervals (depth-based binning, # of ovservations in bin instead of interval of values). Read relevant documentation of the method on the Web.

In the Adult Income dataset, there is an `age` attribute, which is numerical.

<span style="color:red"><b>Write code that will do the following:</b></span>
    
<span style="color:red"><b>1- Apply quantile cutting to the `age` attribute and construct new dataset.</b></span>

<span style="color:red"><b>2- Apply one-hot encoding again to the discretized attribute.</b></span>

In [113]:
# Take another copy of the original dataset
new_income_df2 = encoded_income_df.copy()

# Cut the age attribute into intervals
age_group2 = pd.qcut(new_income_df2.age, q=4 , labels=['1','2','3','4'])



new_income_df2["age_group"] = age_group2.astype('object')
del new_income_df2["age"]

# Try one hot encoding

# Copy the original data


# Select the numeric columns
numeric_subset = new_income_df2.select_dtypes('number')
# Select the categorical columns
categorical_subset = new_income_df2.select_dtypes('object')
# Import feature-engine
from feature_engine.categorical_encoders import OneHotCategoricalEncoder

# Define encoder method
encoder = OneHotCategoricalEncoder()

# Apply encoder to categorical subset, except income
onehot_categorical_subset = encoder.fit_transform(categorical_subset[categorical_subset.columns.drop("income")])

# Beauty of feature-engine encoder method is it already returns a dataframe, so we need not worry about conversions
# Concatenate and reconstruct new dataset
onehot_encoded_data = pd.concat([numeric_subset, onehot_categorical_subset, new_income_df2["income"]], axis = 1)

# display sample 10 records
onehot_encoded_data.sample(10)

,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Local-gov,workclass_?,workclass_Self-emp-not-inc,workclass_Federal-gov,...,native-country_Greece,native-country_Trinadad&Tobago,native-country_Outlying-US(Guam-USVI-etc),native-country_France,native-country_Holand-Netherlands,age_group_1,age_group_3,age_group_2,age_group_4,income
48730,71556,14,0,0,50,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,>50K
37125,108446,10,0,0,40,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,<=50K
17888,102318,9,0,0,40,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,<=50K
37893,173350,9,0,0,30,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,<=50K
34276,126327,12,0,0,40,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,<=50K
3904,178510,13,0,0,45,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,<=50K
33352,278254,9,0,0,40,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,<=50K
37087,129432,13,15024,0,40,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,>50K
25433,175931,9,0,0,40,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,<=50K
9763,378418,9,0,0,30,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,<=50K


## <span style="color:red">Assignment Mini Challenge 4</span>

In the Adult Income dataset, there is an `hours_per_week` attribute, which is numerical.

<span style="color:red"><b>Write code that will do the following:</b></span>

<span style="color:red"><b>1- Transform this attribute into a categorical attribute by dividing the range of values into bins with the following lables: "0-9", "10-19", "20-29", "30-39", "40-49", "50+". Rename the new attribute as `working_hours_categories`. </b></span>

<span style="color:red"><b>2- Perform the chi2 test to find if the discretized attribute `working_hours_categories` and `gender` are correlated.</b></span>


In [129]:
# Take another copy of the original dataset

new_income_df2 = encoded_income_df.copy()
new_income_df2['hours_per_week']=new_income_df2['hours-per-week']
# Cut the age attribute into intervals

hours_per_week_group2 = pd.cut(new_income_df2.hours_per_week, [0,9,19,29,39,49])



new_income_df2["working_hours_categories"] = hours_per_week_group2.astype('object')
del new_income_df2["hours_per_week"]

new_income_df2.sample(10)

contengency_table = pd.crosstab(new_income_df2["working_hours_categories"],new_income_df2["gender"], margins= True)

# Take the previously produced contingency matrix and apply the chi2 test method to it
#st.chi2_contingency(_)
print (st.chi2_contingency(contengency_table))
contengency_table

(1748.748588835073, 0.0, 10, array([[  261.18842726,   438.81157274,   700.        ],
       [  705.58187993,  1185.41812007,  1891.        ],
       [ 1328.32971579,  2231.67028421,  3560.        ],
       [ 2065.62733332,  3470.37266668,  5536.        ],
       [10251.2726437 , 17222.7273563 , 27474.        ],
       [14612.        , 24549.        , 39161.        ]]))


gender,Female,Male,All
working_hours_categories,,,
"(0.0, 9.0]",368,332,700
"(9.0, 19.0]",1030,861,1891
"(19.0, 29.0]",1918,1642,3560
"(29.0, 39.0]",2873,2663,5536
"(39.0, 49.0]",8423,19051,27474
All,14612,24549,39161


#### Aggregation

Now let's try to aggregate numerical values according to a categorical attribute. If there was a time attribute then aggregation could have been performed on different time intervals. For the dataset we have it is sufficient to apply aggregation over the workclass attribute for now. We will use the `groupby` function. Then, it is possible to compute aggregate values (e.g. mean) per workclass group for the numerical attributes.

In [134]:
# We will work with the original dataset income_df
# Group workclass attribute by its categorical values
grouped_income = income_df.groupby(["workclass"])

# Compute mean per group using agg function
grouped_income.agg(np.mean).head()


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
workclass,,,,,,
?,40.141122,187254.773491,9.251161,502.946052,67.634870,31.812433
Federal-gov,42.577514,183590.028631,10.937151,923.287709,108.884078,41.513268
Local-gov,41.676020,190161.134885,11.032207,798.228635,102.124043,40.847258
Never-worked,19.900000,215033.300000,7.500000,0.000000,0.000000,28.900000
Private,36.903144,192669.212499,9.875715,896.135374,80.768478,40.273137


The `agg` function intelligently ignores categorical attributes.

## <span style="color:red">Assignment Mini Challenge 5</span>

<span style="color:red"><b>Why did we not apply the aggregation function on the encoded dataset?</b></span>

``GroupBy`` has its own associated ``aggregate()`` method which allows for more flexibility and aggregate different columns using different aggregation functions in one shot. Look up how this method works, 

<span style="color:red"><b>Write code that uses the ``aggregate()`` to aggregate the `age` column by mean, the `fnlwgt` column by min, the `education-num` by max, the `capital-gain` by max, the `capital-loss` by max, and the `hours-per-week` by mean. </b></span>

In [145]:
# We will work with the original dataset income_df
# Group workclass attribute by its categorical values
grouped_income = income_df.groupby(["workclass"])

# Compute mean per group using agg function
pd.concat([grouped_income["age"].agg(np.mean),
           grouped_income["fnlwgt"].agg(np.min),
           grouped_income["educational-num"].agg(np.max),
           grouped_income["capital-gain"].agg(np.max),
          grouped_income["capital-loss"].agg(np.max),
           grouped_income["hours-per-week"].agg(np.mean)],axis = 1)


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
workclass,,,,,,
?,40.141122,12285,16,99999,4356,31.812433
Federal-gov,42.577514,19522,16,99999,3683,41.513268
Local-gov,41.676020,14878,16,99999,2467,40.847258
Never-worked,19.900000,131593,10,0,0,28.900000
Private,36.903144,13769,16,99999,4356,40.273137
Self-emp-inc,45.799410,21626,16,99999,2824,48.570501
Self-emp-not-inc,45.332470,19896,16,99999,2824,44.395132
State-gov,39.512367,13492,16,99999,3683,39.090863
Without-pay,47.285714,27012,12,4416,1887,33.952381


## <span style="color:red">Assignment Bonus Challenge</span>

<span style="color:red"><b>For the Adult Income Dataset:</b></span>

<span style="color:red"><b>1- Try to come up with at least one indicator feature based on threshold, multiple attributes, or multiple categorical values in an attribute</b></span>

<span style="color:red"><b>2- Try to come up with at least one meaningful interaction feature.</b></span>